In [182]:
import pandas as pd
import numpy as np
import time

import re
#from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing


In [183]:
data = pd.read_json('computers_train_xlarge.json',lines = True)
data.to_csv('train.csv',index = False)
dff = pd.read_csv('train.csv')
dff


,id_left,category_left,cluster_id_left,id_right,category_right,cluster_id_right,label,pair_id,brand_left,brand_right,description_left,description_right,keyValuePairs_left,keyValuePairs_right,price_left,price_right,specTableContent_left,specTableContent_right,title_left,title_right
0,2551242,Computers_and_Accessories,79457,16272671,Computers_and_Accessories,79457,1,2551242#16272671,"""Corsair""@en",NaN,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...",NaN,"{'Memory Type': 'DDR4 (PC4-21300)', 'Capacity'...","{'Categorie': 'Geheugen intern', 'Merk': 'Cors...",NaN,NaN,Memory Type DDR4 (PC4-21300) Capacity 64GB (4...,Categorie Geheugen intern Merk Corsair Produc...,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P..."
1,16757469,Computers_and_Accessories,63553,16476204,Computers_and_Accessories,63553,1,16757469#16476204,"""HP Enterprise""","""HP Enterprise""","""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","{'Category': 'Proliant HardDrive', 'Sub-Catego...","{'Category': 'Proliant HardDrive', 'Sub-Catego...",NaN,"""CAD"", ""$757.74 CAD""",Specifications: Category Proliant HardDrive S...,Specifications: Category Proliant HardDrive S...,"""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null"""
2,232007,Computers_and_Accessories,212917,16442945,Computers_and_Accessories,14319765,0,232007#16442945,"""SanDisk""","""HP Enterprise""","""SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...","""Description:Genuine HPE 1GB FBD PC2-5300(2x5...",NaN,"{'Category': 'Proliant Memory', 'Sub-Category'...","""USD"", ""7.98""","""USD"", ""$69.95""",NaN,Specifications: Category Proliant Memory Sub-...,"""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ..."
3,2066119,Computers_and_Accessories,667772,12411100,Computers_and_Accessories,667772,1,2066119#12411100,NaN,"""KINGSTON""","""DISCO DURO INTERNO SOLIDO HDD SSD""@es",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3..."
4,6656540,Computers_and_Accessories,505997,2639431,Computers_and_Accessories,154118,0,6656540#2639431,NaN,"""Corsair""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68456,6493497,Computers_and_Accessories,222681,16149764,Computers_and_Accessories,502396,0,6493497#16149764,"""HP Enterprise""","""HP Enterprise""","""Description:5 x 300GB 2.5-inch Serial Attache...","""Description:2 x 72GB 2.5-inch Serial Attache...","{'Category': 'Proliant HardDrive', 'Sub-Catego...","{'Category': 'Proliant HardDrive', 'Sub-Catego...",NaN,"""$151.40"", ""USD""",Specifications: Category Proliant HardDrive S...,Specifications: Category Proliant HardDrive S...,"""DG0300FARVV HP 300-GB 6G 10K 2.5 DP SAS"", ""N...","""Null"" ""512743-001 HP 72-GB 6G 15K 2.5 DP SAS"""
68457,17075265,Computers_and_Accessories,944407,17346839,Computers_and_Accessories,825203,0,17075265#17346839,"""Samsung""",NaN,NaN,"""Cost-effective SSD featuring TurboWrite and ...",NaN,NaN,NaN,NaN,NaN,NaN,"""Samsung - 840 EVO 250GB 2.5"" Solid State Dri...","""SSD 750 EVO 2.5” SATA III 120GB ""@en"
68458,16408794,Computers_and_Accessories,144052,3675781,Computers_and_Accessories,51801,0,16408794#3675781,"""INTEL""@fr","""Intel""@en",NaN,"""Quad Core Technology, 3.6GHz clock speed, 8MB...",NaN,"{'Processor Number': 'I7 7700', 'Number of Cor...",NaN,NaN,NaN,Processor Number I7 7700 Number of Cores 4 Nu...,"""Socket H4 1151 - Coffee Lake Core i7-8700K 6...","""7th Generation Intel® Core™ i7 7700 3.6GHz S..."
68459,13925964,Computers_and_Accessories,43816,15659664,Computers_and_Accessories,478188,0,13925964#15659664,NaN,NaN,"""Built to WD's highest standards of quality an...",NaN,NaN,NaN,"""$"", "

In [184]:
dff.columns

Index(['id_left', 'category_left', 'cluster_id_left', 'id_right',
       'category_right', 'cluster_id_right', 'label', 'pair_id', 'brand_left',
       'brand_right', 'description_left', 'description_right',
       'keyValuePairs_left', 'keyValuePairs_right', 'price_left',
       'price_right', 'specTableContent_left', 'specTableContent_right',
       'title_left', 'title_right'],
      dtype='object')

In [185]:
dff.drop(dff.columns[[2,5,6,8,9,12,13,14,15,16,17]],axis=1,inplace =True)
dff

,id_left,category_left,id_right,category_right,pair_id,description_left,description_right,title_left,title_right
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,2551242#16272671,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...",NaN,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P..."
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,16757469#16476204,"""Description:2 x 72GB 2.5-inch Serial Attached...","""Description:10 x 72GB 2.5-inch Serial Attach...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null"""
2,232007,Computers_and_Accessories,16442945,Computers_and_Accessories,232007#16442945,"""SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...","""Description:Genuine HPE 1GB FBD PC2-5300(2x5...","""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ..."
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,2066119#12411100,"""DISCO DURO INTERNO SOLIDO HDD SSD""@es",NaN,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3..."
4,6656540,Computers_and_Accessories,2639431,Computers_and_Accessories,6656540#2639431,NaN,NaN,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4..."
...,...,...,...,...,...,...,...,...,...
68456,6493497,Computers_and_Accessories,16149764,Computers_and_Accessories,6493497#16149764,"""Description:5 x 300GB 2.5-inch Serial Attache...","""Description:2 x 72GB 2.5-inch Serial Attache...","""DG0300FARVV HP 300-GB 6G 10K 2.5 DP SAS"", ""N...","""Null"" ""512743-001 HP 72-GB 6G 15K 2.5 DP SAS"""
68457,17075265,Computers_and_Accessories,17346839,Computers_and_Accessories,17075265#17346839,NaN,"""Cost-effective SSD featuring TurboWrite and ...","""Samsung - 840 EVO 250GB 2.5"" Solid State Dri...","""SSD 750 EVO 2.5” SATA III 120GB ""@en"
68458,16408794,Computers_and_Accessories,3675781,Computers_and_Accessories,16408794#3675781,NaN,"""Quad Core Technology, 3.6GHz clock speed, 8MB...","""Socket H4 1151 - Coffee Lake Core i7-8700K 6...","""7th Generation Intel® Core™ i7 7700 3.6GHz S..."
68459,13925964,Computers_and_Accessories,15659664,Computers_and_Accessories,13925964#15659664,"""Built to WD's highest standards of quality an...",NaN,"""WD Blue WD5000AZLX - hard drive 500 GB SATA ...","""m rock ships""@en-US ""M-ROCK Ships New Camera..."


In [186]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68461 entries, 0 to 68460
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_left            68461 non-null  int64 
 1   category_left      68461 non-null  object
 2   id_right           68461 non-null  int64 
 3   category_right     68461 non-null  object
 4   pair_id            68461 non-null  object
 5   description_left   47460 non-null  object
 6   description_right  48360 non-null  object
 7   title_left         68461 non-null  object
 8   title_right        68461 non-null  object
dtypes: int64(2), object(7)
memory usage: 4.7+ MB


In [187]:
dff['category_left'].value_counts()

Computers_and_Accessories     64958
Other_Electronics              1115
Office_Products                 844
Video_Games                     575
Camera_and_Photo                286
Musical_Instruments             269
Luggage_and_Travel_Gear         256
Tools_and_Home_Improvement      108
Cellphones_and_Accessories       50
Name: category_left, dtype: int64

In [188]:
dff['category_right'].value_counts()

Computers_and_Accessories     66972
Other_Electronics               463
Video_Games                     405
Office_Products                 262
Luggage_and_Travel_Gear         122
Cellphones_and_Accessories      109
Camera_and_Photo                 74
Musical_Instruments              44
Tools_and_Home_Improvement       10
Name: category_right, dtype: int64

In [189]:
dff['id_left'].value_counts()

14619165    43
8364399     42
5411511     41
8167053     40
4927241     40
            ..
6810432      1
16476204     1
10969778     1
5872903      1
4604957      1
Name: id_left, Length: 4287, dtype: int64

In [190]:
(dff['id_left'] < 0).sum()

0

In [191]:
dff['id_right'].value_counts()

7900893     116
12050629    107
7440444     106
6443065      97
13889537     97
           ... 
5199342       1
16567684      1
16123988      1
13661066      1
7867351       1
Name: id_right, Length: 4144, dtype: int64

In [192]:
dff['description_left'].isnull().sum()

21001

# Filling the Null values of Description Columns by Title

In [193]:
dff.description_left.fillna(dff.title_left, inplace = True)
dff['description_left']

0        "DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...
1        "Description:2 x 72GB 2.5-inch Serial Attached...
2         "SDSDJ-1024 BXP 1GB 9p SD Class 2 Secure Digi...
3                  "DISCO DURO INTERNO SOLIDO HDD SSD"@es 
4         "Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...
                               ...                        
68456    "Description:5 x 300GB 2.5-inch Serial Attache...
68457     "Samsung - 840 EVO 250GB 2.5" Solid State Dri...
68458     "Socket H4 1151 - Coffee Lake Core i7-8700K 6...
68459    "Built to WD's highest standards of quality an...
68460    "61 cm 250 cd / m² 1920 x 1080 Pixeles 5 ms LE...
Name: description_left, Length: 68461, dtype: object

In [194]:
dff['description_left'].isnull().sum()

0

In [195]:
dff['description_right'].isnull().sum()

20101

In [196]:
dff.description_right.fillna(dff.title_left, inplace = True)
dff['description_right']

0         "Corsair Vengeance LPX Black 64GB (4x16GB) DD...
1         "Description:10 x 72GB 2.5-inch Serial Attach...
2         "Description:Genuine HPE 1GB FBD PC2-5300(2x5...
3         "DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...
4         "Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...
                               ...                        
68456     "Description:2 x 72GB 2.5-inch Serial Attache...
68457     "Cost-effective SSD featuring TurboWrite and ...
68458    "Quad Core Technology, 3.6GHz clock speed, 8MB...
68459     "WD Blue WD5000AZLX - hard drive 500 GB SATA ...
68460    "Longitud diagonal: 24 "; Tamaño: 16:9; Tecnol...
Name: description_right, Length: 68461, dtype: object

In [197]:
dff['description_right'].isnull().sum()

0

In [198]:
def remove_spaces(text):
    text=text.strip()
    text=text.split()
    return ' '.join(text)

In [199]:
contraction = {'cause':'because',
              'aint': 'am not',
              'aren\'t': 'are not'}

def mapping_replacer(x,dic):
    for words in dic.keys():
        if ' ' + words + ' ' in x:
            x=x.replace(' '+ words +' ' ,' '+dic[words]+' ' )
    return x

In [200]:
#Stemming, lemmetisation and tokenisation
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

nltk.LancasterStemmer
ls = LancasterStemmer()
lem = WordNetLemmatizer()
def lexicon_normalization(text):
    words = word_tokenize(text) 
    
    
    # 1- Stemming
    words_stem = [ls.stem(w) for w in words]
    
    # 2- Lemmatization
    words_lem = [lem.lemmatize(w) for w in words_stem]
    return words_lem

In [201]:
#Removing links, brackets, numbers, punctuations etc.

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('_', '', text)
    text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\'','', text)
    text = re.sub('and', '', text)
    
    
    return text

In [202]:
#Handling stopwords
from collections import Counter
def remove_stopword(text):
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])
    return text

In [203]:
#dc=df['description_right']
#import re
#dc = dc.map(lambda x: re.sub(r'\W+', ' ', str(x)))
#dc = dc.replace(r'\W+', ' ', regex=True)

#dc=dc.apply(lambda x: mapping_replacer(x, contraction))

#dc=dc.apply(lambda x:clean_text(x))

#dc=dc.apply(lambda x: remove_stopword(x))

#dc=dc.apply(lambda x: lexicon_normalization(x))

#dc

In [204]:
import re
df = dff
df['description_right'] = dff['description_right'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df['description_right'] = dff['description_right'].replace(r'\W+', ' ', regex=True)

df['description_right']=dff['description_right'].apply(lambda x: mapping_replacer(x, contraction))

df['description_right']=dff['description_right'].apply(lambda x:clean_text(x))

#df['description_right']=df['description_right'].apply(lambda x: remove_stopword(x))

#df['description_right']=df['description_right'].apply(lambda x: lexicon_normalization(x))

In [205]:
df['description_left'] = dff['description_left'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df['description_left'] = dff['description_left'].replace(r'\W+', ' ', regex=True)

df['description_left']=dff['description_left'].apply(lambda x: mapping_replacer(x, contraction))

df['description_left']=dff['description_left'].apply(lambda x:clean_text(x))

#df['description_left']=df['description_left'].apply(lambda x: remove_stopword(x))

#df['description_left']=df['description_left'].apply(lambda x: lexicon_normalization(x))

In [206]:
df['description_right']

0         corsair vengeance lpx black 64gb 4x16gb ddr4 ...
1         description 10 x 72gb 2 5 inch serial attache...
2         description genuine hpe 1gb fbd pc2 5300 2x51...
3         disco duro interno solido hdd ssd kingston v3...
4         corsair vengeance led 32gb 2 x 16gb ddr4 dram...
                               ...                        
68456     description 2 x 72gb 2 5 inch serial attached...
68457     cost effective ssd featuring turbowrite  rapi...
68458     quad core technology 3 6ghz clock speed 8mb l...
68459     wd blue wd5000azlx hard drive 500 gb sata 6gb...
68460     longitud diagonal 24 tamaño 16 9 tecnología l...
Name: description_right, Length: 68461, dtype: object

In [207]:
df['description_left']

0         ddr4 2666mhz cl16 1 2v xmp 2 0 lifetime warra...
1         description 2 x 72gb 2 5 inch serial attached...
2         sdsdj 1024 bxp 1gb 9p sd class 2 secure digit...
3                    disco duro interno solido hdd ssd es 
4         corsair vengeance led 32gb 2 x 16gb ddr4 dram...
                               ...                        
68456     description 5 x 300gb 2 5 inch serial attache...
68457     samsung 840 evo 250gb 2 5 solid state drive d...
68458     socket h4 1151 coffee lake core i7 8700k 6 co...
68459     built to wd s highest stards of quality  reli...
68460     61 cm 250 cd m² 1920 x 1080 pixeles 5 ms led es 
Name: description_left, Length: 68461, dtype: object

In [208]:
# Creating Target column 

def id_target(n):
    if(n<0):
        return 0
    else:
        return 1

df['target'] = df['id_right'].apply(id_target)
df

,id_left,category_left,id_right,category_right,pair_id,description_left,description_right,title_left,title_right,target
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,2551242#16272671,ddr4 2666mhz cl16 1 2v xmp 2 0 lifetime warra...,corsair vengeance lpx black 64gb 4x16gb ddr4 ...,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P...",1
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,16757469#16476204,description 2 x 72gb 2 5 inch serial attached...,description 10 x 72gb 2 5 inch serial attache...,"""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null""",1
2,232007,Computers_and_Accessories,16442945,Computers_and_Accessories,232007#16442945,sdsdj 1024 bxp 1gb 9p sd class 2 secure digit...,description genuine hpe 1gb fbd pc2 5300 2x51...,"""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ...",1
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,2066119#12411100,disco duro interno solido hdd ssd es,disco duro interno solido hdd ssd kingston v3...,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3...",1
4,6656540,Computers_and_Accessories,2639431,Computers_and_Accessories,6656540#2639431,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4...",1
...,...,...,...,...,...,...,...,...,...,...
68456,6493497,Computers_and_Accessories,16149764,Computers_and_Accessories,6493497#16149764,description 5 x 300gb 2 5 inch serial attache...,description 2 x 72gb 2 5 inch serial attached...,"""DG0300FARVV HP 300-GB 6G 10K 2.5 DP SAS"", ""N...","""Null"" ""512743-001 HP 72-GB 6G 15K 2.5 DP SAS""",1
68457,17075265,Computers_and_Accessories,17346839,Computers_and_Accessories,17075265#17346839,samsung 840 evo 250gb 2 5 solid state drive d...,cost effective ssd featuring turbowrite rapi...,"""Samsung - 840 EVO 250GB 2.5"" Solid State Dri...","""SSD 750 EVO 2.5” SATA III 120GB ""@en",1
68458,16408794,Computers_and_Accessories,3675781,Computers_and_Accessories,16408794#3675781,socket h4 1151 coffee lake core i7 8700k 6 co...,quad core technology 3 6ghz clock speed 8mb l...,"""Socket H4 1151 - Coffee Lake Core i7-8700K 6...","""7th Generation Intel® Core™ i7 7700 3.6GHz S...",1
68459,13925964,Computers_and_Accessories,15659664,Computers_and_Accessories,13925964#15659664,built to wd s highest stards of quality reli...,wd blue wd5000azlx hard drive 500 gb sata 6gb...,"""WD Blue WD5000AZLX - hard drive 500 GB SATA ...","""m rock ships""@en-US ""M-ROCK Ships New Camera...",1


In [209]:
# Counting matched and not matched Products

df['target'].value_counts()

#0 for negative id_right values
#1 for positive id_right values

1    68461
Name: target, dtype: int64

In [210]:
# Comparing category left and category right

df['category_match'] = np.where(df['category_left'] == df['category_right'],1,0)
df.head()

#1-Match / 0-Unmatch

,id_left,category_left,id_right,category_right,pair_id,description_left,description_right,title_left,title_right,target,category_match
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,2551242#16272671,ddr4 2666mhz cl16 1 2v xmp 2 0 lifetime warra...,corsair vengeance lpx black 64gb 4x16gb ddr4 ...,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P...",1,1
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,16757469#16476204,description 2 x 72gb 2 5 inch serial attached...,description 10 x 72gb 2 5 inch serial attache...,"""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null""",1,1
2,232007,Computers_and_Accessories,16442945,Computers_and_Accessories,232007#16442945,sdsdj 1024 bxp 1gb 9p sd class 2 secure digit...,description genuine hpe 1gb fbd pc2 5300 2x51...,"""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ...",1,1
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,2066119#12411100,disco duro interno solido hdd ssd es,disco duro interno solido hdd ssd kingston v3...,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3...",1,1
4,6656540,Computers_and_Accessories,2639431,Computers_and_Accessories,6656540#2639431,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4...",1,1


In [211]:
df['category_match'].value_counts()

1    65018
0     3443
Name: category_match, dtype: int64

In [212]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68461 entries, 0 to 68460
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_left            68461 non-null  int64 
 1   category_left      68461 non-null  object
 2   id_right           68461 non-null  int64 
 3   category_right     68461 non-null  object
 4   pair_id            68461 non-null  object
 5   description_left   68461 non-null  object
 6   description_right  68461 non-null  object
 7   title_left         68461 non-null  object
 8   title_right        68461 non-null  object
 9   target             68461 non-null  int64 
 10  category_match     68461 non-null  int32 
dtypes: int32(1), int64(3), object(7)
memory usage: 5.5+ MB


In [213]:
df.drop(df.index[df['category_match'] == 0],inplace = True)

In [214]:
df['category_match'].value_counts()

1    65018
Name: category_match, dtype: int64

In [215]:
newdf=df.copy()
newdf.head()

,id_left,category_left,id_right,category_right,pair_id,description_left,description_right,title_left,title_right,target,category_match
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,2551242#16272671,ddr4 2666mhz cl16 1 2v xmp 2 0 lifetime warra...,corsair vengeance lpx black 64gb 4x16gb ddr4 ...,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P...",1,1
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,16757469#16476204,description 2 x 72gb 2 5 inch serial attached...,description 10 x 72gb 2 5 inch serial attache...,"""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null""",1,1
2,232007,Computers_and_Accessories,16442945,Computers_and_Accessories,232007#16442945,sdsdj 1024 bxp 1gb 9p sd class 2 secure digit...,description genuine hpe 1gb fbd pc2 5300 2x51...,"""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ...",1,1
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,2066119#12411100,disco duro interno solido hdd ssd es,disco duro interno solido hdd ssd kingston v3...,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3...",1,1
4,6656540,Computers_and_Accessories,2639431,Computers_and_Accessories,6656540#2639431,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4...",1,1


In [216]:
# Applying Maching Numbers function to match the products Features 

def matching_numbers(description_right, description_left):

    description_right = set(re.findall(r'[0-9]+', description_right))
    description_left = set(re.findall(r'[0-9]+', description_left))    
    union = description_right.union(description_left)
    intersection = description_right.intersection(description_left)

    if len(description_right)==0 and len(description_left) == 0:
        return 1
    else:
        return (len(intersection)/ len(union))

In [217]:
# Implementing Levenshtein Text similarity 


import jellyfish as jf
def engineer_features(df):
    
    df['description_left'] = df['description_left'].str.lower()
    df['description_right'] = df['description_right'].str.lower()

    df['levenshtein_distance'] = df.apply(
    lambda x: jf.levenshtein_distance(x['description_right'], 
                                      x['description_left']), axis=1)

    df['matching_numbers'] = df.apply(
    lambda x: matching_numbers(x['description_right'], 
                               x['description_left']), axis=1)

    df['matching_numbers_log'] = (df['matching_numbers']+1).apply(np.log)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(value=0, inplace=True)

    return df

In [218]:
# All Required Features 

df.head()

,id_left,category_left,id_right,category_right,pair_id,description_left,description_right,title_left,title_right,target,category_match
0,2551242,Computers_and_Accessories,16272671,Computers_and_Accessories,2551242#16272671,ddr4 2666mhz cl16 1 2v xmp 2 0 lifetime warra...,corsair vengeance lpx black 64gb 4x16gb ddr4 ...,"""Corsair Vengeance LPX Black 64GB (4x16GB) DD...","""Corsair Vengeance LPX CMK64GX4M4A2666C16 - P...",1,1
1,16757469,Computers_and_Accessories,16476204,Computers_and_Accessories,16757469#16476204,description 2 x 72gb 2 5 inch serial attached...,description 10 x 72gb 2 5 inch serial attache...,"""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"", ""Nu...","""DH0072BALWL HP 72-GB 3G 15K 2.5 DP SAS"" ""Null""",1,1
2,232007,Computers_and_Accessories,16442945,Computers_and_Accessories,232007#16442945,sdsdj 1024 bxp 1gb 9p sd class 2 secure digit...,description genuine hpe 1gb fbd pc2 5300 2x51...,"""SanDisk SDSDJ-1024 BXP 1GB 9p SD Class 2 Sec...","""397409-B21 HP 1GB (2x512MB) PC2-5300 SDRAM"" ...",1,1
3,2066119,Computers_and_Accessories,12411100,Computers_and_Accessories,2066119#12411100,disco duro interno solido hdd ssd es,disco duro interno solido hdd ssd kingston v3...,"""DISCO DURO INTERNO SOLIDO HDD SSD KINGSTON V...","""DISCO DURO SSD Kingston Technology SSDNow V3...",1,1
4,6656540,Computers_and_Accessories,2639431,Computers_and_Accessories,6656540#2639431,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,"""Corsair Vengeance LED 32GB (2 x 16GB) DDR4 D...","""Corsair - Vengeance LPX 32GB (2 x 16GB) DDR4...",1,1


In [219]:
df = engineer_features(df)
df = df[['description_left','description_right','levenshtein_distance','matching_numbers']]
df

,description_left,description_right,levenshtein_distance,matching_numbers
0,ddr4 2666mhz cl16 1 2v xmp 2 0 lifetime warra...,corsair vengeance lpx black 64gb 4x16gb ddr4 ...,96,0.375000
1,description 2 x 72gb 2 5 inch serial attached...,description 10 x 72gb 2 5 inch serial attache...,28,0.933333
2,sdsdj 1024 bxp 1gb 9p sd class 2 secure digit...,description genuine hpe 1gb fbd pc2 5300 2x51...,83,0.250000
3,disco duro interno solido hdd ssd es,disco duro interno solido hdd ssd kingston v3...,66,0.000000
4,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,corsair vengeance led 32gb 2 x 16gb ddr4 dram...,0,1.000000
...,...,...,...,...
68456,description 5 x 300gb 2 5 inch serial attache...,description 2 x 72gb 2 5 inch serial attached...,95,0.350000
68457,samsung 840 evo 250gb 2 5 solid state drive d...,cost effective ssd featuring turbowrite rapi...,108,0.000000
68458,socket h4 1151 coffee lake core i7 8700k 6 co...,quad core technology 3 6ghz clock speed 8mb l...,102,0.300000
68459,built to wd s highest stards of quality reli...,wd blue wd5000azlx hard drive 500 gb sata 6gb...,886,0.000000


In [220]:
df=df.sort_values(by=['levenshtein_distance'], ascending=[False])
df

,description_left,description_right,levenshtein_distance,matching_numbers
66134,div id productdescriptioncontent style box si...,null tr,45893,0.000000
58120,div id productdescriptioncontent style box si...,disco duro 3 5 7200rpm sata 6gb s 64mb cache es,45853,0.041667
24715,seagate desktop 7200 14 2tb sata3 64mb es 64m...,div id productdescriptioncontent style box si...,45852,0.041667
24999,wd blue wd10ezex 1tb prijzen nl wd10ezex twea...,div id productdescriptioncontent style box si...,45849,0.028571
65772,wd red wd30efrx 3tb desktop internal hard dri...,div id productdescriptioncontent style box si...,45847,0.014085
...,...,...,...,...
46120,corsair air series sp120 pwm quiet edition tw...,corsair air series sp120 pwm quiet edition tw...,0,1.000000
17031,fm2 a4 7300 2core 4ghz 1mb radeon hd8470d 65w...,fm2 a4 7300 2core 4ghz 1mb radeon hd8470d 65w...,0,1.000000
17032,intel core i5 4670 comprar intel i5 4670 powe...,intel core i5 4670 comprar intel i5 4670 powe...,0,1.000000
25135,windows 10 pro 64 intel core i7 7700hq memori...,windows 10 pro 64 intel core i7 7700hq memori...,0,1.000000


In [221]:
# Examining the mean ,count and max values of columns

df.describe()

,levenshtein_distance,matching_numbers
count,65018.000000,65018.000000
mean,295.046602,0.329379
std,1346.641992,0.346787
min,0.000000,0.000000
25%,59.000000,0.071429
50%,103.000000,0.200000
75%,245.000000,0.454545
max,45893.000000,1.000000


In [222]:
df['match'] = np.where(((df['levenshtein_distance']<70) & (df['matching_numbers']>0.4)),1,0)
df['match'].value_counts()

df

,description_left,description_right,levenshtein_distance,matching_numbers,match
66134,div id productdescriptioncontent style box si...,null tr,45893,0.000000,0
58120,div id productdescriptioncontent style box si...,disco duro 3 5 7200rpm sata 6gb s 64mb cache es,45853,0.041667,0
24715,seagate desktop 7200 14 2tb sata3 64mb es 64m...,div id productdescriptioncontent style box si...,45852,0.041667,0
24999,wd blue wd10ezex 1tb prijzen nl wd10ezex twea...,div id productdescriptioncontent style box si...,45849,0.028571,0
65772,wd red wd30efrx 3tb desktop internal hard dri...,div id productdescriptioncontent style box si...,45847,0.014085,0
...,...,...,...,...,...
46120,corsair air series sp120 pwm quiet edition tw...,corsair air series sp120 pwm quiet edition tw...,0,1.000000,1
17031,fm2 a4 7300 2core 4ghz 1mb radeon hd8470d 65w...,fm2 a4 7300 2core 4ghz 1mb radeon hd8470d 65w...,0,1.000000,1
17032,intel core i5 4670 comprar intel i5 4670 powe...,intel core i5 4670 comprar intel i5 4670 powe...,0,1.000000,1
25135,windows 10 pro 64 intel core i7 7700hq memori...,windows 10 pro 64 intel core i7 7700hq memori...,0,1.000000,1


In [223]:
# Applying Train test Split For Finding Model Accuracy

X = df[['levenshtein_distance',
       'matching_numbers']].values
y = df['match'].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
# y_train

In [281]:
data1 = pd.read_json('testset_1500.json',lines = True)
data1.to_csv('test.csv',index = False)
df_test = pd.read_csv('test.csv')
df_test

,id_left,category_left,title_left,description_left,brand_left,price_left,keyValuePairs_left,specTableContent_left,id_right,category_right,title_right,description_right,brand_right,price_right,keyValuePairs_right,specTableContent_right,pair_id
0,1656083,Computers_and_Accessories,"""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz C...","""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz Ca...","""Corsair""",NaN,"{'Item Code :': 'MEC-S3-8GDV169', 'Item :': 'C...",Item Code : MEC-S3-8GDV169 Item : Corsair Ven...,4082241,Computers_and_Accessories,"""Crucial 8GB Single DDR3L 1600 MT/s (PC3L-128...","""In short, 204-pin small outline dual inline m...","""Crucial""@en-US",NaN,NaN,NaN,1656083#4082241
1,1220565,Computers_and_Accessories,"""488039-B21 HP X5470 3.33GHz ML370 G5"", ""Null...","""Description:Intel Xeon X5470 ML370 G5 (3.33GH...","""HP Enterprise""",NaN,"{'Category': 'Proliant Processor', 'Sub-Catego...",Specifications: Category Proliant Processor S...,4215042,Computers_and_Accessories,"""349237-B21 HP 80-GB 1.5G 7.2K 3.5 SATA HDD""","""Description:80GB Hot-Plug Serial ATA (SATA) ...","""HP Enterprise""","""CAD"", "" $111.74 CAD""","{'Manufacturer': 'DL320', 'Category': 'Hard Dr...",Specifications: Manufacturer DL320 Category ...,1220565#4215042
2,10969978,Computers_and_Accessories,"""Corsair Air Series SP120 PWM High Performanc...","""Size: 120mm x 25mm, Operating Voltage: 7V-12V...","""Corsair""@en",NaN,"{'Dimension': '120mm x 25mm', 'Voltage Rating'...",Dimension 120mm x 25mm Voltage Rating 7V – 12...,-128473760,Computers_and_Accessories,"""SP120 PWM High Pressure Fan 4 pin, Single Pa...","""The wide body low-pitch blades of the SP120 f...","""Corsair""@en",NaN,NaN,NaN,10969978#-128473760
3,1749942,Computers_and_Accessories,"""Speck Flaptop Sleeve for MacBook Air 13""""@en...",NaN,NaN,NaN,NaN,NaN,-88566620,Computers_and_Accessories,"""FlapTop notebook sleeve"" "" sleeve - 77496-5...","""Intuitive closure uses a responsive flex zone...",NaN,"""$"", ""31.99""",NaN,NaN,1749942#-88566620
4,403680,Computers_and_Accessories,"""Asus M5A99X PRO R2.0 AMD 990X (Socket AM3+) ...","""AMD 990FX Chipset, x4 DDR3, x4 PCI-E x16, x1 ...","""Asus""@en",NaN,NaN,NaN,-91536280,Computers_and_Accessories,"""Buy Online | AMD A99FX PRO R2.0 Motherboard ...","""\nA99FX PRO R2.0 Motherboard\n""@en-US",NaN,NaN,NaN,Weight 3 kg LAN / IEEE GB/N PCIe x16/x4/ x1-m...,403680#-91536280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,72675,Computers_and_Accessories,"""Null"" ""495942-B21 HP E5506 2.13GHz ML370 G6""","""Description:Intel Xeon E5506 DL/ML370 G6(2.1...","""HP Enterprise""","""USD"", ""$139.95""","{'Category': 'Proliant Processor', 'Sub-Catego...",Specifications: Category Proliant Processor S...,14412683,Computers_and_Accessories,"""583383-B21 HP Xeon E5506 2.13GHz DL170h G6"" ...","""Description:HP Intel Xeon E5506 DL170h G6(2....","""HP Enterprise""","""$195.95"", ""USD""","{'Category': 'Proliant Processor', 'Sub-Catego...",Specifications: Category Proliant Processor S...,72675#14412683
1496,2365521,Computers_and_Accessories,"""AMD Ryzen Threadripper 1950X Box""@es","""Procesador CPU 16C/32T unlocked, Auto-OC, 3.4...",NaN,NaN,NaN,NaN,16049825,Computers_and_Accessories,"""AMD Ryzen 5 1400 Box""@es","""Procesador CPU 4C/8T unlocked, 3.2/3.4Ghz, Ze...",NaN,NaN,NaN,NaN,2365521#16049825
1497,2807522,Computers_and_Accessories,"""WD Red 6TB NAS Hard Disk Drive - 5400 RPM Cl...","""WD Red is the only hard drive designed for NA...","""Western Digital""@en-US",NaN,NaN,NaN,15298918,Video_Games,"""LOGITECH 910-001105 PERFORMANCE MOUSE MX USB...","""OverviewDARKFIELD LASER TRACKINGï¿½ PROVIDES ...","""LOGITECH""",NaN,NaN,NaN,2807522#15298918
1498,268014,Computers_and_Accessories,"""Samsung 850 EVO 250 GB Internal SSD"" "" Samsu...","""The Samsung SSD 850 EVO elevates the everyday...",NaN,"""$"", ""109.99""",NaN,NaN,8837825,Computers_and_Accessories,"""Samsung 850 EVO 500GB MZ-75E500B/EU""@es","""Disco Solido SSD, 500GB de capacidad, 2.5 

In [282]:
df_test.columns

Index(['id_left', 'category_left', 'title_left', 'description_left',
       'brand_left', 'price_left', 'keyValuePairs_left',
       'specTableContent_left', 'id_right', 'category_right', 'title_right',
       'description_right', 'brand_right', 'price_right',
       'keyValuePairs_right', 'specTableContent_right', 'pair_id'],
      dtype='object')

In [283]:
df_test.drop(df_test.columns[[4,5,6,7,12,13,14,15]],axis=1,inplace =True)
df_test

,id_left,category_left,title_left,description_left,id_right,category_right,title_right,description_right,pair_id
0,1656083,Computers_and_Accessories,"""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz C...","""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz Ca...",4082241,Computers_and_Accessories,"""Crucial 8GB Single DDR3L 1600 MT/s (PC3L-128...","""In short, 204-pin small outline dual inline m...",1656083#4082241
1,1220565,Computers_and_Accessories,"""488039-B21 HP X5470 3.33GHz ML370 G5"", ""Null...","""Description:Intel Xeon X5470 ML370 G5 (3.33GH...",4215042,Computers_and_Accessories,"""349237-B21 HP 80-GB 1.5G 7.2K 3.5 SATA HDD""","""Description:80GB Hot-Plug Serial ATA (SATA) ...",1220565#4215042
2,10969978,Computers_and_Accessories,"""Corsair Air Series SP120 PWM High Performanc...","""Size: 120mm x 25mm, Operating Voltage: 7V-12V...",-128473760,Computers_and_Accessories,"""SP120 PWM High Pressure Fan 4 pin, Single Pa...","""The wide body low-pitch blades of the SP120 f...",10969978#-128473760
3,1749942,Computers_and_Accessories,"""Speck Flaptop Sleeve for MacBook Air 13""""@en...",NaN,-88566620,Computers_and_Accessories,"""FlapTop notebook sleeve"" "" sleeve - 77496-5...","""Intuitive closure uses a responsive flex zone...",1749942#-88566620
4,403680,Computers_and_Accessories,"""Asus M5A99X PRO R2.0 AMD 990X (Socket AM3+) ...","""AMD 990FX Chipset, x4 DDR3, x4 PCI-E x16, x1 ...",-91536280,Computers_and_Accessories,"""Buy Online | AMD A99FX PRO R2.0 Motherboard ...","""\nA99FX PRO R2.0 Motherboard\n""@en-US",403680#-91536280
...,...,...,...,...,...,...,...,...,...
1495,72675,Computers_and_Accessories,"""Null"" ""495942-B21 HP E5506 2.13GHz ML370 G6""","""Description:Intel Xeon E5506 DL/ML370 G6(2.1...",14412683,Computers_and_Accessories,"""583383-B21 HP Xeon E5506 2.13GHz DL170h G6"" ...","""Description:HP Intel Xeon E5506 DL170h G6(2....",72675#14412683
1496,2365521,Computers_and_Accessories,"""AMD Ryzen Threadripper 1950X Box""@es","""Procesador CPU 16C/32T unlocked, Auto-OC, 3.4...",16049825,Computers_and_Accessories,"""AMD Ryzen 5 1400 Box""@es","""Procesador CPU 4C/8T unlocked, 3.2/3.4Ghz, Ze...",2365521#16049825
1497,2807522,Computers_and_Accessories,"""WD Red 6TB NAS Hard Disk Drive - 5400 RPM Cl...","""WD Red is the only hard drive designed for NA...",15298918,Video_Games,"""LOGITECH 910-001105 PERFORMANCE MOUSE MX USB...","""OverviewDARKFIELD LASER TRACKINGï¿½ PROVIDES ...",2807522#15298918
1498,268014,Computers_and_Accessories,"""Samsung 850 EVO 250 GB Internal SSD"" "" Samsu...","""The Samsung SSD 850 EVO elevates the everyday...",8837825,Computers_and_Accessories,"""Samsung 850 EVO 500GB MZ-75E500B/EU""@es","""Disco Solido SSD, 500GB de capacidad, 2.5 SAT...",268014#8837825


In [284]:
df_test['description_right'].isnull().sum()

219

In [285]:
df_test.description_right.fillna(df_test.title_left, inplace = True)
df_test['description_right']

0       "In short, 204-pin small outline dual inline m...
1        "Description:80GB Hot-Plug Serial ATA (SATA) ...
2       "The wide body low-pitch blades of the SP120 f...
3       "Intuitive closure uses a responsive flex zone...
4                 "\nA99FX PRO R2.0 Motherboard\n"@en-US 
                              ...                        
1495     "Description:HP Intel Xeon E5506 DL170h G6(2....
1496    "Procesador CPU 4C/8T unlocked, 3.2/3.4Ghz, Ze...
1497    "OverviewDARKFIELD LASER TRACKINGï¿½ PROVIDES ...
1498    "Disco Solido SSD, 500GB de capacidad, 2.5 SAT...
1499    "Plug and Play for high performance with Intel...
Name: description_right, Length: 1500, dtype: object

In [286]:
df_test['description_right'].isnull().sum()

0

In [287]:
df_test['description_left'].isnull().sum()

167

In [288]:
df_test.description_left.fillna(df_test.title_left, inplace = True)
df_test['description_left']

0       "Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz Ca...
1       "Description:Intel Xeon X5470 ML370 G5 (3.33GH...
2       "Size: 120mm x 25mm, Operating Voltage: 7V-12V...
3        "Speck Flaptop Sleeve for MacBook Air 13""@en...
4       "AMD 990FX Chipset, x4 DDR3, x4 PCI-E x16, x1 ...
                              ...                        
1495     "Description:Intel Xeon E5506 DL/ML370 G6(2.1...
1496    "Procesador CPU 16C/32T unlocked, Auto-OC, 3.4...
1497    "WD Red is the only hard drive designed for NA...
1498    "The Samsung SSD 850 EVO elevates the everyday...
1499    "HyperX FURY FURY Memory Black 64GB DDR4 2133M...
Name: description_left, Length: 1500, dtype: object

In [289]:
df_test['description_left'].isnull().sum()

0

In [292]:
def id_target(n):
    if(n<0):
        return 0
    else:
        return 1
    

In [293]:
df_test['target'] = df_test['id_right'].apply(id_target)
df_test.head()

,id_left,category_left,title_left,description_left,id_right,category_right,title_right,description_right,pair_id,target
0,1656083,Computers_and_Accessories,"""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz C...","""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz Ca...",4082241,Computers_and_Accessories,"""Crucial 8GB Single DDR3L 1600 MT/s (PC3L-128...","""In short, 204-pin small outline dual inline m...",1656083#4082241,1
1,1220565,Computers_and_Accessories,"""488039-B21 HP X5470 3.33GHz ML370 G5"", ""Null...","""Description:Intel Xeon X5470 ML370 G5 (3.33GH...",4215042,Computers_and_Accessories,"""349237-B21 HP 80-GB 1.5G 7.2K 3.5 SATA HDD""","""Description:80GB Hot-Plug Serial ATA (SATA) ...",1220565#4215042,1
2,10969978,Computers_and_Accessories,"""Corsair Air Series SP120 PWM High Performanc...","""Size: 120mm x 25mm, Operating Voltage: 7V-12V...",-128473760,Computers_and_Accessories,"""SP120 PWM High Pressure Fan 4 pin, Single Pa...","""The wide body low-pitch blades of the SP120 f...",10969978#-128473760,0
3,1749942,Computers_and_Accessories,"""Speck Flaptop Sleeve for MacBook Air 13""""@en...","""Speck Flaptop Sleeve for MacBook Air 13""""@en...",-88566620,Computers_and_Accessories,"""FlapTop notebook sleeve"" "" sleeve - 77496-5...","""Intuitive closure uses a responsive flex zone...",1749942#-88566620,0
4,403680,Computers_and_Accessories,"""Asus M5A99X PRO R2.0 AMD 990X (Socket AM3+) ...","""AMD 990FX Chipset, x4 DDR3, x4 PCI-E x16, x1 ...",-91536280,Computers_and_Accessories,"""Buy Online | AMD A99FX PRO R2.0 Motherboard ...","""\nA99FX PRO R2.0 Motherboard\n""@en-US",403680#-91536280,0


In [295]:
df_test['target'].value_counts()


1    1300
0     200
Name: target, dtype: int64

In [297]:
df_test['category_match'] = np.where(df_test['category_left'] == df_test['category_right'],1,0)
df_test.head()

,id_left,category_left,title_left,description_left,id_right,category_right,title_right,description_right,pair_id,target,category_match
0,1656083,Computers_and_Accessories,"""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz C...","""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz Ca...",4082241,Computers_and_Accessories,"""Crucial 8GB Single DDR3L 1600 MT/s (PC3L-128...","""In short, 204-pin small outline dual inline m...",1656083#4082241,1,1
1,1220565,Computers_and_Accessories,"""488039-B21 HP X5470 3.33GHz ML370 G5"", ""Null...","""Description:Intel Xeon X5470 ML370 G5 (3.33GH...",4215042,Computers_and_Accessories,"""349237-B21 HP 80-GB 1.5G 7.2K 3.5 SATA HDD""","""Description:80GB Hot-Plug Serial ATA (SATA) ...",1220565#4215042,1,1
2,10969978,Computers_and_Accessories,"""Corsair Air Series SP120 PWM High Performanc...","""Size: 120mm x 25mm, Operating Voltage: 7V-12V...",-128473760,Computers_and_Accessories,"""SP120 PWM High Pressure Fan 4 pin, Single Pa...","""The wide body low-pitch blades of the SP120 f...",10969978#-128473760,0,1
3,1749942,Computers_and_Accessories,"""Speck Flaptop Sleeve for MacBook Air 13""""@en...","""Speck Flaptop Sleeve for MacBook Air 13""""@en...",-88566620,Computers_and_Accessories,"""FlapTop notebook sleeve"" "" sleeve - 77496-5...","""Intuitive closure uses a responsive flex zone...",1749942#-88566620,0,1
4,403680,Computers_and_Accessories,"""Asus M5A99X PRO R2.0 AMD 990X (Socket AM3+) ...","""AMD 990FX Chipset, x4 DDR3, x4 PCI-E x16, x1 ...",-91536280,Computers_and_Accessories,"""Buy Online | AMD A99FX PRO R2.0 Motherboard ...","""\nA99FX PRO R2.0 Motherboard\n""@en-US",403680#-91536280,0,1


In [299]:
df_test.drop(df_test.index[df_test['category_match'] == 0],inplace = True)
df_test.head()

,id_left,category_left,title_left,description_left,id_right,category_right,title_right,description_right,pair_id,target,category_match
0,1656083,Computers_and_Accessories,"""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz C...","""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz Ca...",4082241,Computers_and_Accessories,"""Crucial 8GB Single DDR3L 1600 MT/s (PC3L-128...","""In short, 204-pin small outline dual inline m...",1656083#4082241,1,1
1,1220565,Computers_and_Accessories,"""488039-B21 HP X5470 3.33GHz ML370 G5"", ""Null...","""Description:Intel Xeon X5470 ML370 G5 (3.33GH...",4215042,Computers_and_Accessories,"""349237-B21 HP 80-GB 1.5G 7.2K 3.5 SATA HDD""","""Description:80GB Hot-Plug Serial ATA (SATA) ...",1220565#4215042,1,1
2,10969978,Computers_and_Accessories,"""Corsair Air Series SP120 PWM High Performanc...","""Size: 120mm x 25mm, Operating Voltage: 7V-12V...",-128473760,Computers_and_Accessories,"""SP120 PWM High Pressure Fan 4 pin, Single Pa...","""The wide body low-pitch blades of the SP120 f...",10969978#-128473760,0,1
3,1749942,Computers_and_Accessories,"""Speck Flaptop Sleeve for MacBook Air 13""""@en...","""Speck Flaptop Sleeve for MacBook Air 13""""@en...",-88566620,Computers_and_Accessories,"""FlapTop notebook sleeve"" "" sleeve - 77496-5...","""Intuitive closure uses a responsive flex zone...",1749942#-88566620,0,1
4,403680,Computers_and_Accessories,"""Asus M5A99X PRO R2.0 AMD 990X (Socket AM3+) ...","""AMD 990FX Chipset, x4 DDR3, x4 PCI-E x16, x1 ...",-91536280,Computers_and_Accessories,"""Buy Online | AMD A99FX PRO R2.0 Motherboard ...","""\nA99FX PRO R2.0 Motherboard\n""@en-US",403680#-91536280,0,1


In [300]:
df_test['description_right'] = df_test['description_right'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df_test['description_right'] = df_test['description_right'].replace(r'\W+', ' ', regex=True)

df_test['description_right']=df_test['description_right'].apply(lambda x: mapping_replacer(x, contraction))

df_test['description_right']=df_test['description_right'].apply(lambda x:clean_text(x))

#df_test['description_right']=df_test['description_right'].apply(lambda x: remove_stopword(x))

#df['description_right']=df['description_right'].apply(lambda x: lexicon_normalization(x))

In [301]:
df_test['description_left'] = df_test['description_left'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df_test['description_left'] = df_test['description_left'].replace(r'\W+', ' ', regex=True)

df_test['description_left']=df_test['description_left'].apply(lambda x: mapping_replacer(x, contraction))

df_test['description_left']=df_test['description_left'].apply(lambda x:clean_text(x))

#df_test['description_left']=df_test['description_left'].apply(lambda x: remove_stopword(x))

#df['description_right']=df['description_right'].apply(lambda x: lexicon_normalization(x))

In [305]:
newdf=df_test.copy()
newdf

,id_left,category_left,title_left,description_left,id_right,category_right,title_right,description_right,pair_id,target,category_match,levenshtein_distance,matching_numbers,matching_numbers_log
0,1656083,Computers_and_Accessories,"""Corsair Vengeance 8Gb (2x4Gb) DDR3 1600MHz C...",corsair vengeance 8gb 2x4gb ddr3 1600mhz cas9...,4082241,Computers_and_Accessories,"""Crucial 8GB Single DDR3L 1600 MT/s (PC3L-128...",in short 204 pin small outline dual inline me...,1656083#4082241,1,1,256,0.090909,0.087011
1,1220565,Computers_and_Accessories,"""488039-B21 HP X5470 3.33GHz ML370 G5"", ""Null...",description intel xeon x5470 ml370 g5 3 33ghz...,4215042,Computers_and_Accessories,"""349237-B21 HP 80-GB 1.5G 7.2K 3.5 SATA HDD""",description 80gb hot plug serial ata sata 1 5...,1220565#4215042,1,1,333,0.093750,0.089612
2,10969978,Computers_and_Accessories,"""Corsair Air Series SP120 PWM High Performanc...",size 120mm x 25mm operating voltage 7v 12v pe...,-128473760,Computers_and_Accessories,"""SP120 PWM High Pressure Fan 4 pin, Single Pa...",the wide body low pitch blades of the sp120 f...,10969978#-128473760,0,1,126,0.125000,0.117783
3,1749942,Computers_and_Accessories,"""Speck Flaptop Sleeve for MacBook Air 13""""@en...",speck flaptop sleeve for macbook air 13 en 13...,-88566620,Computers_and_Accessories,"""FlapTop notebook sleeve"" "" sleeve - 77496-5...",intuitive closure uses a responsive flex zone...,1749942#-88566620,0,1,124,0.000000,0.000000
4,403680,Computers_and_Accessories,"""Asus M5A99X PRO R2.0 AMD 990X (Socket AM3+) ...",amd 990fx chipset x4 ddr3 x4 pci e x16 x1 pci...,-91536280,Computers_and_Accessories,"""Buy Online | AMD A99FX PRO R2.0 Motherboard ...",a99fx pro r2 0 motherboard en us,403680#-91536280,0,1,137,0.181818,0.167054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,1455319,Computers_and_Accessories,"""TP-Link 150Mbps Wireless PCI Express Adapter...",tp link tl wn781nd is designed to provide a c...,8027162,Computers_and_Accessories,"""TP-Link 5-Port Gigabit Ethernet Desktop Swit...",the tl sg1005d 5 port gigabit rack mount swit...,1455319#8027162,1,1,213,0.000000,0.000000
1495,72675,Computers_and_Accessories,"""Null"" ""495942-B21 HP E5506 2.13GHz ML370 G6""",description intel xeon e5506 dl ml370 g6 2 13...,14412683,Computers_and_Accessories,"""583383-B21 HP Xeon E5506 2.13GHz DL170h G6"" ...",description hp intel xeon e5506 dl170h g6 2 1...,72675#14412683,1,1,18,0.583333,0.459532
1496,2365521,Computers_and_Accessories,"""AMD Ryzen Threadripper 1950X Box""@es",procesador cpu 16c 32t unlocked auto oc 3 4 4...,16049825,Computers_and_Accessories,"""AMD Ryzen 5 1400 Box""@es",procesador cpu 4c 8t unlocked 3 2 3 4ghz zen ...,2365521#16049825,1,1,35,0.416667,0.348307
1498,268014,Computers_and_Accessories,"""Samsung 850 EVO 250 GB Internal SSD"" "" Samsu...",the samsung ssd 850 evo elevates the everyday...,8837825,Computers_and_Accessories,"""Samsung 850 EVO 500GB MZ-75E500B/EU""@es",disco solido ssd 500gb de capacidad 2 5 sata ...,268014#8837825,1,1,289,0.000000,0.000000


In [306]:
df_test = engineer_features(df_test)
df_test = df_test[['description_left','description_right','levenshtein_distance','matching_numbers','target']]
df_test

,description_left,description_right,levenshtein_distance,matching_numbers,target
0,corsair vengeance 8gb 2x4gb ddr3 1600mhz cas9...,in short 204 pin small outline dual inline me...,256,0.090909,1
1,description intel xeon x5470 ml370 g5 3 33ghz...,description 80gb hot plug serial ata sata 1 5...,333,0.093750,1
2,size 120mm x 25mm operating voltage 7v 12v pe...,the wide body low pitch blades of the sp120 f...,126,0.125000,0
3,speck flaptop sleeve for macbook air 13 en 13...,intuitive closure uses a responsive flex zone...,124,0.000000,0
4,amd 990fx chipset x4 ddr3 x4 pci e x16 x1 pci...,a99fx pro r2 0 motherboard en us,137,0.181818,0
...,...,...,...,...,...
1494,tp link tl wn781nd is designed to provide a c...,the tl sg1005d 5 port gigabit rack mount swit...,213,0.000000,1
1495,description intel xeon e5506 dl ml370 g6 2 13...,description hp intel xeon e5506 dl170h g6 2 1...,18,0.583333,1
1496,procesador cpu 16c 32t unlocked auto oc 3 4 4...,procesador cpu 4c 8t unlocked 3 2 3 4ghz zen ...,35,0.416667,1
1498,the samsung ssd 850 evo elevates the everyday...,disco solido ssd 500gb de capacidad 2 5 sata ...,289,0.000000,1


In [308]:
df_test[df_test.columns[1:]].corr()['target'][:].sort_values(ascending=False)


target                  1.000000
matching_numbers        0.036718
levenshtein_distance   -0.063098
Name: target, dtype: float64

In [309]:
X_train=df[['levenshtein_distance','matching_numbers']]
X_test=df_test[['levenshtein_distance','matching_numbers']]
y_train=df['match']
y_test=df_test['target']
y_train

66134    0
58120    0
24715    0
24999    0
65772    0
        ..
46120    1
17031    1
17032    1
25135    1
36254    1
Name: match, Length: 65018, dtype: int32

In [310]:
# Creating Function For Finding Confusion Matrix

def get_confusion_matrix_values(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    return(cm[0][0], cm[0][1], cm[1][0], cm[1][1])

In [311]:
# Model Building: Decision Tree and Random Forest 

classifiers = {
    "DecisionTreeClassifier":DecisionTreeClassifier(),
    "RandomForestClassifier":RandomForestClassifier(n_estimators=100,max_depth=8),
    
}

df_results = pd.DataFrame(columns=['model', 'accuracy', 'precision',
                                   'recall','true_pos','false_pos',
                                   'true_neg','false_neg'])

for key in classifiers:

    classifier = classifiers[key]
    model = classifier.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    classification = classification_report(y_test, y_pred, zero_division=0)
    tp, fp, fn, tn = get_confusion_matrix_values(y_test, y_pred)

    row = {'model': key,
           'accuracy': accuracy,
           'precision': precision,
           'recall': recall,
           'f1': f1,
           'true_pos': tp,
           'false_pos': fp,
           'true_neg': tn,
           'false_neg': fn,
          }
    df_results = df_results.append(row, ignore_index=True)
    
    

df_results.head(10)


,model,accuracy,precision,recall,true_pos,false_pos,true_neg,false_neg,f1
0,DecisionTreeClassifier,0.35242,0.93188,0.269929,175,25,342,925,0.418605
1,RandomForestClassifier,0.35242,0.93188,0.269929,175,25,342,925,0.418605


In [312]:

from xgboost import XGBClassifier
from xgboost import XGBClassifier
from sklearn.datasets import load_wine

model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, grow_policy=None,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_bin=None, max_cat_to_onehot=None,
              max_delta_step=0, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', predictor=None, random_state=0,
              reg_alpha=0, ...)

In [313]:
y_pred = model.predict(X_test)
y_pred 

array([0, 0, 0, ..., 1, 0, 0])

In [314]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.35241990456714384

In [315]:

classifier=RandomForestClassifier(n_estimators=100)
model = classifier.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.35241990456714384

In [316]:
# Estimating Results in Binary

results = pd.DataFrame(data={'predictions': y_pred, 'actual': y_test})
results['result'] = np.where(results['predictions']==results['actual'], 1, 0)
results

,predictions,actual,result
0,0,1,0
1,0,1,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
1494,0,1,0
1495,1,1,1
1496,1,1,1
1498,0,1,0


In [317]:
# Final Results with Match and Not Matched Classification

results['predictions'].replace(0, 'Not match',inplace=True)
results['predictions'].replace(1, 'Match',inplace=True)

results['actual'].replace(0, 'Not Match',inplace=True)
results['actual'].replace(1, 'Match',inplace=True)

results['result'].replace(0, 'False',inplace=True)
results['result'].replace(1, 'True',inplace=True)
results

,predictions,actual,result
0,Not match,Match,False
1,Not match,Match,False
2,Not match,Not Match,True
3,Not match,Not Match,True
4,Not match,Not Match,True
...,...,...,...
1494,Not match,Match,False
1495,Match,Match,True
1496,Match,Match,True
1498,Not match,Match,False
